In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../../data/chocolate_encoded.csv')


In [3]:
df.head()


,Unnamed: 0,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,...,x3_2.0,x3_2.25,x3_2.5,x3_2.6,x3_2.75,x3_3.0,x3_3.25,x3_3.5,x3_3.75,x3_4.0
0,0,5.0,0.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,15.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,15.0,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,15.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,15.0,0.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df = df.dropna()

In [5]:
from sklearn.model_selection import train_test_split
y= df[['x3_1.0','x3_1.5','x3_1.75','x3_2.0','x3_2.25','x3_2.5','x3_2.75','x3_3.0','x3_3.5','x3_3.75','x3_4.0']]
x = df.drop(['x3_1.0','x3_1.5','x3_1.75','x3_2.0','x3_2.25','x3_2.5','x3_2.75','x3_3.0','x3_3.5','x3_3.75','x3_4.0'],axis=1)


In [6]:
x_tr, x_test, y_tr, y_test = train_test_split(x,y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, random_state=57)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

In [8]:
base_KNN = KNeighborsClassifier()

base_KNN.fit(x_train, y_train)

train_pred = base_KNN.predict(x_train)
val_pred = base_KNN.predict(x_val)



In [10]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

0.3146796431467964
0.21897810218978103


Baseline KNN classifier, preforms decent with accuracy, but a grid search could likely improve it at least some

In [11]:
from sklearn.model_selection import GridSearchCV

GS_KNN = KNeighborsClassifier()

#tried wieghts distance, lead to overfitting so it's been removed as an option
pram_grid = {'n_neighbors': [1,50,100,200,500], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[5,30,100],'p':[1,2]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [5, 30, 100],
                         'n_neighbors': [1, 50, 100, 200, 500], 'p': [1, 2],
                         'weights': ['uniform']},
             scoring='accuracy')

In [12]:
GSCV.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 30,
 'n_neighbors': 1,
 'p': 1,
 'weights': 'uniform'}

In [14]:
train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

1.0
0.24087591240875914


Definitely imporved the model, but there's definently work to be done, so let's try and refine further

In [15]:
GS_KNN = KNeighborsClassifier()


pram_grid = {'n_neighbors': [1,5,10,15,20,25,30], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[20,30,40],'p':[1]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [20, 30, 40],
                         'n_neighbors': [1, 5, 10, 15, 20, 25, 30], 'p': [1],
                         'weights': ['uniform']},
             scoring='accuracy')

In [16]:
GSCV.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 20,
 'n_neighbors': 5,
 'p': 1,
 'weights': 'uniform'}

In [17]:
train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

0.30413625304136255
0.22871046228710462


Ok so much smaller improvement, but n_neighbors boeign 1 seems super wrong, let's try narrowing down again and seeing what happens

In [18]:
GS_KNN = KNeighborsClassifier()


pram_grid3 = {'n_neighbors': [3,4,5,6,7], 'weights' : ['uniform'], 'algorithm':['ball_tree'],'leaf_size':[15,20,25],'p':[1]}

GSCV3 = GridSearchCV(GS_KNN,pram_grid3,scoring = 'accuracy')

GSCV3.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree'], 'leaf_size': [15, 20, 25],
                         'n_neighbors': [3, 4, 5, 6, 7], 'p': [1],
                         'weights': ['uniform']},
             scoring='accuracy')

In [19]:
GSCV3.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 20,
 'n_neighbors': 3,
 'p': 1,
 'weights': 'uniform'}

In [20]:
train_pred3 = GSCV3.predict(x_train)
val_pred3 = GSCV3.predict(x_val)


train_acc3 = accuracy_score(y_train,train_pred3)
val_acc3 = accuracy_score(y_val,val_pred3)

print(train_acc3)
print(val_acc3)

0.43390105433901055
0.22384428223844283


So accuracy has lowered, but this seems overfit so let's keep trying to optimize where we can and see what happens

In [21]:
GS_KNN = KNeighborsClassifier()


pram_grid4 = {'n_neighbors': [2,3,4], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[18,19,20,21,22],'p':[1,2]}

GSCV4 = GridSearchCV(GS_KNN,pram_grid4,scoring = 'accuracy')

GSCV4.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [18, 19, 20, 21, 22],
                         'n_neighbors': [2, 3, 4], 'p': [1, 2],
                         'weights': ['uniform']},
             scoring='accuracy')

In [22]:
GSCV4.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 18,
 'n_neighbors': 3,
 'p': 2,
 'weights': 'uniform'}

In [23]:
train_pred4 = GSCV4.predict(x_train)
val_pred4 = GSCV4.predict(x_val)


train_acc4 = accuracy_score(y_train,train_pred4)
val_acc4 = accuracy_score(y_val,val_pred4)

print(train_acc4)
print(val_acc4)

0.43390105433901055
0.19708029197080293


So It's really feeling like this data just isn't working right now, so easy quick improvement, let's try binning the targets into larger groups and see if that helps

In [7]:
df.head()

,Unnamed: 0,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,...,x3_2.0,x3_2.25,x3_2.5,x3_2.6,x3_2.75,x3_3.0,x3_3.25,x3_3.5,x3_3.75,x3_4.0
0,0,5.0,0.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,15.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,15.0,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,15.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,15.0,0.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:

df['y_1.0'] = df['x3_1.0']
df['y_1.5'] = df['x3_1.5'] + df['x3_1.75']
df['y_2.0'] = df['x3_2.0'] + df['x3_2.25']
df['y_2.5'] = df['x3_2.5'] + df['x3_2.75']
df['y_3.0'] = df['x3_3.0'] + df['x3_3.25']
df['y_3.5'] = df['x3_3.5'] + df['x3_3.75']
df['y_4.0'] = df['x3_4.0']

df = df.drop(['x3_1.0','x3_1.5','x3_1.75','x3_2.0','x3_2.25','x3_2.5','x3_2.75','x3_3.0','x3_3.5','x3_3.75','x3_4.0'],axis=1)

In [11]:
df.head()

,Unnamed: 0,REF,Cocoa Percent,fruity,sandy,honey,spice,sour,nut,berry,...,x2_Vietnam,x3_2.6,x3_3.25,y_1.0,y_1.5,y_2.0,y_2.5,y_3.0,y_3.5,y_4.0
0,0,5.0,0.71,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,15.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,15.0,0.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,15.0,0.70,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,15.0,0.82,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [12]:
y= df[['y_1.0','y_1.5','y_2.0','y_2.5','y_3.0','y_3.5','y_4.0']]
x = df.drop(['y_1.0','y_1.5','y_2.0','y_2.5','y_3.0','y_3.5','y_4.0'], axis =1)
x_tr, x_test, y_tr, y_test = train_test_split(x,y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, random_state=57)

In [13]:
from sklearn.model_selection import GridSearchCV

GS_KNN = KNeighborsClassifier()

#tried wieghts distance, lead to overfitting so it's been removed as an option
pram_grid = {'n_neighbors': [1,50,100,200,500], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[5,30,100],'p':[1,2]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [5, 30, 100],
                         'n_neighbors': [1, 50, 100, 200, 500], 'p': [1, 2],
                         'weights': ['uniform']},
             scoring='accuracy')

In [14]:
GSCV.best_params_

{'algorithm': 'brute',
 'leaf_size': 5,
 'n_neighbors': 1,
 'p': 1,
 'weights': 'uniform'}

In [16]:
from sklearn.metrics import accuracy_score


train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

1.0
0.3795620437956204


This is significantly better than before binning, but let's tune these hyper parameters up and see if it can't be improved

In [17]:
from sklearn.model_selection import GridSearchCV

GS_KNN = KNeighborsClassifier()

#tried wieghts distance, lead to overfitting so it's been removed as an option
pram_grid = {'n_neighbors': [1,10,20,30], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[5,10,15,20],'p':[1,2]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [5, 10, 15, 20],
                         'n_neighbors': [1, 10, 20, 30], 'p': [1, 2],
                         'weights': ['uniform']},
             scoring='accuracy')

In [18]:
GSCV.best_params_

{'algorithm': 'brute',
 'leaf_size': 5,
 'n_neighbors': 1,
 'p': 1,
 'weights': 'uniform'}

In [19]:
train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

1.0
0.3795620437956204


So this ended up being exactly the same as the previous grid search but it has technically narrowed down on our target so on to the next model

In [20]:
from sklearn.model_selection import GridSearchCV

GS_KNN = KNeighborsClassifier()

#tried wieghts distance, lead to overfitting so it's been removed as an option
pram_grid = {'n_neighbors': [1,3,5,7], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[1,3,5,7,9],'p':[1,2]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [1, 3, 5, 7, 9],
                         'n_neighbors': [1, 3, 5, 7], 'p': [1, 2],
                         'weights': ['uniform']},
             scoring='accuracy')

In [21]:
GSCV.best_params_

{'algorithm': 'kd_tree',
 'leaf_size': 1,
 'n_neighbors': 1,
 'p': 1,
 'weights': 'uniform'}

In [22]:
train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

1.0
0.3844282238442822


So there's and overfitting issue here, let's try messing with things to see if we can't fix that

In [34]:

GS_KNN = KNeighborsClassifier()

#tried wieghts distance, lead to overfitting so it's been removed as an option
pram_grid = {'n_neighbors': [2,3,4], 'weights' : ['uniform'], 'algorithm':['ball_tree','kd_tree','brute'],'leaf_size':[1,2,3,5,7],'p':[1,2]}

GSCV = GridSearchCV(GS_KNN,pram_grid,scoring = 'accuracy')

GSCV.fit(x_train,y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [1, 2, 3, 5, 7], 'n_neighbors': [2, 3, 4],
                         'p': [1, 2], 'weights': ['uniform']},
             scoring='accuracy')

In [35]:
GSCV.best_params_

{'algorithm': 'ball_tree',
 'leaf_size': 5,
 'n_neighbors': 3,
 'p': 1,
 'weights': 'uniform'}

In [36]:
train_pred = GSCV.predict(x_train)
val_pred = GSCV.predict(x_val)


train_acc = accuracy_score(y_train,train_pred)
val_acc = accuracy_score(y_val,val_pred)

print(train_acc)
print(val_acc)

0.5823195458231955
0.30656934306569344
